# Leakage Randomized Benchmarking (RB) on Qiskit

#For Leakage RB:<br>
Leakage errors occur when qubits go out fo computaional basis<br>
Example:<br>
IBM superconductor qubits are based on an "anharmonic LC oscillator" [1]. In this circuit, |0> and |1> energy states form computational basises, whereas |2> state has a certain population possibility during operations, which we refer to leakage error. In silicon spin (exchange-only) qubit, the leakage error is caused by magnetic gradients due to hyperfine interactions with nuclear spins [2]. Noted that Qiskit, as a device-agnostic language, have recently implemented on trapped-ion, we therefore aim to build a software automated package to characterize the effects both from leakage and seepage errors. [3] <br>
[1] https://blog.qutech.nl/index.php/2017/08/13/how-to-make-artificial-atoms-out-of-electrical-circuits-part-ii-circuit-quantum-electrodynamics-and-the-transmon/ <br>
[2] “Quantifying error and leakage in an encoded Si/SiGe triple-dot qubit” Nat. Nanotech., 14, 747 (2019)<br>
[3] "Quantification and Characterization of Leakage Errors" Phys. Rev. A 97, 032306 (2018)

In [1]:
from IPython.display import Image
from IPython.core.display import HTML
display(HTML("<table><tr><td><img src='https://blog.qutech.nl/wp-content/uploads/2017/05/circuit_energy_levels-01-985x1024.png', width=450, height=400></td><td><img src='https://media.springernature.com/m312/springer-static/image/art%3A10.1038%2Fs41565-019-0500-4/MediaObjects/41565_2019_500_Fig1_HTML.png?as=webp', width=450, height=400></td></tr></table>"))

,


For standard RB
Sequence of $m$ Clifford gates

$\rho-\boxed{ C_0 }-\boxed{ C_1 }- ... -\boxed{ C_m }-\boxed{M}$

In the ideal case, applying an inverted clifford gates sequence should give us an identity:<br>
<br>
$\rho-\boxed{ C_0 }-\boxed{ C_1 }- ... -\boxed{ C_m }-\boxed{ C_m^{-1}...C_0^{-1} }-\boxed{M}$
<br><br>
equals<br>
<br>
$\rho-\boxed{ Id }-\boxed{M}$

However, the real computation is noisy; we can model the noise as:<br>
$\rho-\boxed{ C_0 }$-
<font color=red >$\boxed{\Lambda_0}$</font>-
$\boxed{C_1}$-
<font color=red >$\boxed{\Lambda_1}$</font>-
...-
$\boxed{C_m^{-1}...C_0^{-1}}-\boxed{\Lambda_{m+1}}-\boxed{M}$

Consequently, fidelities between ideal and realistic cases can be fitted to the model
<br>

$\overline{F}_m=Ap^m+B$
<br>

under time-independent and gate-independent assumptions on each $\Lambda$.

For the case where Leakage errors occurs during the RB Sequence  <br>
$\rho-\boxed{ C_0 }$- <font color=red >$\boxed{\Lambda_0}$</font>- $\boxed{C_1}$- <font color=red >$\boxed{\Lambda_1}$</font>-...- $\boxed{C_m^{-1}...C_0^{-1}}-\boxed{\Lambda_{m+1}}-\boxed{M}$

Leakage RB fit: <br>
$$p_\mathbb{I}(m)=A_0+B_0\lambda_1^m$$

Our Leakage RB circits on IBM qiskit
Prob of 00 <br>
![](https://imgur.com/5YeD0l7.png)
Prob of 01 <br>
![](https://imgur.com/m9F51NJ.png)
Prob of 10 <br>
![](https://imgur.com/w5IUo0l.png)
Prob of 11 <br>
![](https://imgur.com/lK63DV1.png)


# Run the Leakage RB in Qiskit
There is a wondeful tutorial about RB using qiskit([https://community.qiskit.org/textbook/ch-quantum-hardware/randomized-benchmarking.html](https://community.qiskit.org/textbook/ch-quantum-hardware/randomized-benchmarking.html)).
We added some codes on **ignis**, rough flow of running program is similar to RB[1].

### How to Install 
Wether python nor Anaconda, you can install qiskit by run below command:

**$ pip install qiskit**

If install finished, there shoud be **ignis/vertification/randomized_benchmarking/circuits.py** in qiskit folder.
If you find that file, please replace that to new program.

Then, let's start running Leakage RBM:  
(We should import some qiskit classes first)

In [1]:
%load_ext autoreload
%autoreload 2

#Import general libraries (needed for functions)
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

#Import the RB Functions
import qiskit.ignis.verification.randomized_benchmarking as rb


#Import Qiskit classes 
import qiskit
from qiskit.providers.aer import noise
import qiskit.quantum_info as qi
from qiskit.providers.aer.noise.errors.standard_errors import depolarizing_error, thermal_relaxation_error

## Define Noize Model on qasm_simulator

In [2]:
noise_model = noise.NoiseModel()
# Depolarizing_error
dp = 0.005 
noise_model.add_all_qubit_quantum_error(depolarizing_error(dp, 1), ['u1', 'u2', 'u3'])
noise_model.add_all_qubit_quantum_error(depolarizing_error(2*dp, 2), 'cx')

backend = qiskit.Aer.get_backend('qasm_simulator')

## Define Leakage error model
We created simple leakage noize model, which we can use in 1 or 2-qubit leakage RB(Sample code is 2-qubit Leakage RB, so explain assumeing 2-qubit).   
The Architecture of new noize model is 3-qubit, first and second qubit are the space for calculateing.
Third qubit is Qubit as Enviroment. If leakage is leaked from calculating space, Qubit as Enviroment will be influenced.  
Then, state of Qubit as Enviroment become |1> from |0>. So if any leakage, the measurment result of third qubit become |1>.

In codes, there are 4 parameters of noize:
* p_depolarizing0
* p_depolarizing1
* p_leakage0
* p_leakage1

First two are the depolarizing noize probability of calculating space 0-qubit and 1-qubit.  
Last two are the probability which changes the third qubit state into |1> from |0> per gates.

In [3]:
def leakage_error(qubit_error, p_leakage):
    no_leak = qi.Choi(qubit_error).expand(qi.Operator([[1,0], [0, 1]]))
    leak = qi.Choi(qubit_error).expand(qi.Operator([[0,1],[1,0]]))
    
    return (1 - p_leakage) * no_leak + p_leakage * leak

p_depol1 = 0.01
p_depol2 = 0.01
p_leakage1 = 0.01
p_leakage2 = 0.01
error_1q_leakage = leakage_error(noise.errors.depolarizing_error(p_depol1, 1), p_leakage1)
error_2q_leakage = leakage_error(noise.errors.depolarizing_error(p_depol2, 2), p_leakage2)

noise_leakage = noise.NoiseModel()
noise_leakage.add_nonlocal_quantum_error(error_1q_leakage, ['u1', 'u2', 'u3'], [0], [0, 2])
noise_leakage.add_nonlocal_quantum_error(error_1q_leakage, ['u1', 'u2', 'u3'], [1], [1, 2])
noise_leakage.add_nonlocal_quantum_error(error_2q_leakage, ['cx'], [0, 1], [0, 1, 2])
noise_leakage.add_nonlocal_quantum_error(error_2q_leakage, ['cx'], [1, 0], [1, 0, 2])

## Define RB sequences

In [4]:
#Generate RB circuits (2Q RB)

#number of qubits
nQ=2 


rb_opts = {}


#Number of Cliffords in the sequence
length_vector = [1, 10, 20, 50, 75, 100, 125, 150, 175, 200]
rb_opts['length_vector'] = length_vector

#Number of seeds (random sequences)
nseeds = 5
rb_opts['nseeds'] = nseeds

## Default pattern

rb_pattern = [[0,1]]
rb_opts['rb_pattern'] = rb_pattern ## ここは一つだと動かないので、あとで治す


is_leakage = True
rb_opts['is_leakage'] = is_leakage


print(rb_opts)
"""
rb_circs: circuits for randomized benchmarking

xdata: the length of clifford group
"""
rb_circs, xdata= rb.randomized_benchmarking_seq(**rb_opts)

{'length_vector': [1, 10, 20, 50, 75, 100, 125, 150, 175, 200], 'nseeds': 5, 'rb_pattern': [[0, 1]], 'is_leakage': True}


Here, we passed a new argument **is_leakage** to function **randomized_benchmarking_seq()**.

This argument is True/False flag, which set False in default.
According to this argument, randomized benchmarking will change:

    True  : Leakage Randomized Benchmarking  
    False : Randomized Benchmarking  

In [ ]:
# Create the RB fitter
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] 
shots = 100
qobj_list = []

if is_leakage:

    counts_list = []
    for rb_seed,rb_circ_seed in enumerate(rb_circs):

        print('################### Compiling seed %d ###################'%rb_seed)

        count_seed = []
        for meas_x in range(2**nQ):
            print("meas_x: %d"%meas_x)
            _rb_circ_seed_xmeas = []
            _len_v = len(length_vector)

            #  クリフォードの長さごとの
            count_measx = []
            for i in range(_len_v):
                _rb_circ_seed_xmeas.append(rb_circ_seed[i][meas_x])

                new_rb_circ_seed = qiskit.compiler.transpile(_rb_circ_seed_xmeas, basis_gates=basis_gates)
                qobj = qiskit.compiler.assemble(new_rb_circ_seed, shots=shots)
                print('Simulating seed %d'%rb_seed)

                ## run on simulator 
                job = backend.run(qobj, noise_model=noise_leakage, backend_options={'max_parallel_experiments': 0})
                qobj_list.append(qobj)

                counts = job.result().get_counts(i)
                count_measx.append(counts)
                print("count_measx = ", count_measx)
                
            count_seed.append(count_measx)
            print("count_seed = ", count_seed)
        counts_list.append(count_seed)

################### Compiling seed 0 ###################
meas_x: 0
Simulating seed 0
count_measx =  [{'1 00': 1, '0 10': 31, '0 11': 25, '0 00': 20, '1 10': 1, '0 01': 22}]
Simulating seed 0
count_measx =  [{'1 00': 1, '0 10': 31, '0 11': 25, '0 00': 20, '1 10': 1, '0 01': 22}, {'1 00': 13, '0 11': 18, '1 10': 8, '0 01': 19, '0 10': 12, '1 11': 4, '1 01': 7, '0 00': 19}]
Simulating seed 0
count_measx =  [{'1 00': 1, '0 10': 31, '0 11': 25, '0 00': 20, '1 10': 1, '0 01': 22}, {'1 00': 13, '0 11': 18, '1 10': 8, '0 01': 19, '0 10': 12, '1 11': 4, '1 01': 7, '0 00': 19}, {'1 00': 16, '0 11': 17, '1 10': 9, '0 01': 8, '0 10': 6, '1 11': 17, '1 01': 5, '0 00': 22}]
Simulating seed 0
count_measx =  [{'1 00': 1, '0 10': 31, '0 11': 25, '0 00': 20, '1 10': 1, '0 01': 22}, {'1 00': 13, '0 11': 18, '1 10': 8, '0 01': 19, '0 10': 12, '1 11': 4, '1 01': 7, '0 00': 19}, {'1 00': 16, '0 11': 17, '1 10': 9, '0 01': 8, '0 10': 6, '1 11': 17, '1 01': 5, '0 00': 22}, {'1 00': 20, '0 11': 4, '1 10': 12, 

Simulating seed 0
count_measx =  [{'0 10': 20, '0 11': 27, '1 11': 2, '0 00': 23, '1 01': 1, '1 10': 3, '0 01': 24}, {'1 00': 11, '0 11': 19, '1 10': 8, '0 01': 21, '0 10': 13, '1 11': 3, '1 01': 11, '0 00': 14}, {'1 00': 6, '0 11': 7, '1 10': 15, '0 01': 13, '0 10': 26, '1 11': 3, '1 01': 15, '0 00': 15}, {'1 00': 15, '0 11': 11, '1 10': 9, '0 01': 14, '0 10': 14, '1 11': 10, '1 01': 10, '0 00': 17}, {'1 00': 5, '0 11': 16, '1 10': 8, '0 01': 17, '0 10': 12, '1 11': 16, '1 01': 14, '0 00': 12}, {'1 00': 9, '0 11': 14, '1 10': 19, '0 01': 6, '0 10': 10, '1 11': 12, '1 01': 20, '0 00': 10}, {'1 00': 11, '0 11': 19, '1 10': 13, '0 01': 11, '0 10': 17, '1 11': 12, '1 01': 9, '0 00': 8}]
Simulating seed 0
count_measx =  [{'0 10': 20, '0 11': 27, '1 11': 2, '0 00': 23, '1 01': 1, '1 10': 3, '0 01': 24}, {'1 00': 11, '0 11': 19, '1 10': 8, '0 01': 21, '0 10': 13, '1 11': 3, '1 01': 11, '0 00': 14}, {'1 00': 6, '0 11': 7, '1 10': 15, '0 01': 13, '0 10': 26, '1 11': 3, '1 01': 15, '0 00': 15},

Simulating seed 0
count_measx =  [{'1 00': 1, '0 10': 27, '0 11': 29, '0 00': 25, '1 01': 1, '1 10': 2, '0 01': 15}, {'1 00': 6, '0 11': 15, '1 10': 8, '0 01': 20, '0 10': 15, '1 11': 10, '1 01': 6, '0 00': 20}, {'1 00': 7, '0 11': 7, '1 10': 23, '0 01': 20, '0 10': 18, '1 11': 7, '1 01': 13, '0 00': 5}, {'1 00': 10, '0 11': 14, '1 10': 11, '0 01': 9, '0 10': 15, '1 11': 12, '1 01': 18, '0 00': 11}, {'1 00': 12, '0 11': 15, '1 10': 12, '0 01': 15, '0 10': 15, '1 11': 13, '1 01': 8, '0 00': 10}, {'1 00': 13, '0 11': 14, '1 10': 9, '0 01': 15, '0 10': 12, '1 11': 10, '1 01': 13, '0 00': 14}, {'1 00': 10, '0 11': 12, '1 10': 14, '0 01': 11, '0 10': 8, '1 11': 14, '1 01': 19, '0 00': 12}, {'1 00': 18, '0 11': 9, '1 10': 16, '0 01': 7, '0 10': 10, '1 11': 12, '1 01': 15, '0 00': 13}, {'1 00': 18, '0 11': 10, '1 10': 10, '0 01': 10, '0 10': 13, '1 11': 9, '1 01': 12, '0 00': 18}]
Simulating seed 0
count_measx =  [{'1 00': 1, '0 10': 27, '0 11': 29, '0 00': 25, '1 01': 1, '1 10': 2, '0 01': 1

Simulating seed 0
count_measx =  [{'1 00': 3, '0 10': 23, '0 11': 23, '0 00': 17, '1 01': 1, '1 10': 1, '0 01': 32}, {'1 00': 6, '0 11': 14, '1 10': 10, '0 01': 27, '0 10': 20, '1 11': 9, '1 01': 6, '0 00': 8}, {'1 00': 9, '0 11': 17, '1 10': 6, '0 01': 9, '0 10': 12, '1 11': 20, '1 01': 5, '0 00': 22}, {'1 00': 13, '0 11': 19, '1 10': 12, '0 01': 11, '0 10': 13, '1 11': 8, '1 01': 14, '0 00': 10}, {'1 00': 11, '0 11': 19, '1 10': 13, '0 01': 12, '0 10': 11, '1 11': 10, '1 01': 10, '0 00': 14}, {'1 00': 13, '0 11': 17, '1 10': 11, '0 01': 10, '0 10': 8, '1 11': 7, '1 01': 18, '0 00': 16}, {'1 00': 20, '0 11': 12, '1 10': 5, '0 01': 11, '0 10': 10, '1 11': 16, '1 01': 13, '0 00': 13}, {'1 00': 21, '0 11': 13, '1 10': 7, '0 01': 13, '0 10': 14, '1 11': 9, '1 01': 10, '0 00': 13}, {'1 00': 14, '0 11': 15, '1 10': 10, '0 01': 12, '0 10': 13, '1 11': 12, '1 01': 16, '0 00': 8}]
Simulating seed 0
count_measx =  [{'1 00': 3, '0 10': 23, '0 11': 23, '0 00': 17, '1 01': 1, '1 10': 1, '0 01': 32

Simulating seed 1
count_measx =  [{'0 11': 48, '1 11': 3, '1 01': 1, '0 00': 1, '0 01': 47}, {'1 00': 10, '0 11': 3, '1 10': 12, '0 01': 4, '0 10': 32, '1 11': 4, '1 01': 8, '0 00': 27}, {'1 00': 12, '0 11': 16, '1 10': 13, '0 01': 20, '0 10': 12, '1 11': 9, '1 01': 10, '0 00': 8}, {'1 00': 4, '0 11': 19, '1 10': 12, '0 01': 16, '0 10': 15, '1 11': 12, '1 01': 14, '0 00': 8}, {'1 00': 9, '0 11': 11, '1 10': 17, '0 01': 11, '0 10': 14, '1 11': 10, '1 01': 12, '0 00': 16}, {'1 00': 12, '0 11': 14, '1 10': 15, '0 01': 13, '0 10': 10, '1 11': 12, '1 01': 13, '0 00': 11}, {'1 00': 13, '0 11': 14, '1 10': 14, '0 01': 14, '0 10': 10, '1 11': 9, '1 01': 15, '0 00': 11}, {'1 00': 11, '0 11': 17, '1 10': 13, '0 01': 12, '0 10': 10, '1 11': 8, '1 01': 14, '0 00': 15}]
Simulating seed 1
count_measx =  [{'0 11': 48, '1 11': 3, '1 01': 1, '0 00': 1, '0 01': 47}, {'1 00': 10, '0 11': 3, '1 10': 12, '0 01': 4, '0 10': 32, '1 11': 4, '1 01': 8, '0 00': 27}, {'1 00': 12, '0 11': 16, '1 10': 13, '0 01': 

In [5]:
print('rb_circs[0][0][1]')
rb_circs[0][0][1].draw()

rb_circs[0][0][1]


In [6]:
print('rb_circs[0][0][2]')
rb_circs[0][0][2].draw()

rb_circs[0][0][2]


In [7]:
print('rb_circs[0][0][3]')
rb_circs[0][0][3].draw()

rb_circs[0][0][3]


You could see the 4type circuits like above description of Leaked RB.  

## Execute Leakage RB
Let's execute the leakage RB.  
We have to calculate the probability of the state |000> for 4 types.
Probability is calculated by **(count|000>) / (shots number)**.
At last, sum up to all probability, and get the **Probability in-the Computational-subspace**.

In [ ]:
leakprob_list = []
len_vec = len(length_vector)

for clif_len in range(len_vec):
    mean_seed = []
    for seed in range(nseeds):

        p00 = counts_list[seed][0][clif_len].get('0 00', 0) / shots
        p01 = counts_list[seed][1][clif_len].get('0 00', 0) / shots
        p10 = counts_list[seed][2][clif_len].get('0 00', 0) / shots
        p11 = counts_list[seed][3][clif_len].get('0 00', 0) / shots
        p_id = p00 + p01 + p10 + p11

        mean_seed.append(p_id)
    print(mean_seed)
        
    leakprob_list.append(np.median(mean_seed))
print("leakprob = ", leakprob_list)

## plot

In [2]:
plt.figure(figsize=(8, 6))

fig,ax = plt.subplots(figsize=(10, 10))


## set grid
ax.grid(which = "major", axis = "y", color = "gray", alpha = 0.8, linestyle = "--", linewidth = 0.5)
ax.grid(which = "major", axis = "x", color = "gray", alpha = 0.8, linestyle = "--", linewidth = 0.5)

## set axis
plt.ylabel("Grond State Population", fontsize= 32)
ax.set_yticks(np.linspace(0, 1, 11))
ax.set_yticklabels(["0.0", "0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9", "1.0"], 
                   fontsize= 24)
plt.ylim(0,1)


plt.xlabel("Clifford length", fontsize= 32)
ax.set_xticks(np.linspace(0, 200, 9))

ax.set_xticklabels(["0", "25", "50", "75", "100", "125", "150", "175", "200"],
                   fontsize= 20)
plt.xlim(-0.1,200.1)


## data
plt.plot(length_vector, leakprob_list, color='#EF5350', )

NameError: name 'plt' is not defined

In [ ]:
# Create the RB fitter
backend = qiskit.Aer.get_backend('qasm_simulator')
basis_gates = ['u1','u2','u3','cx'] 
shots = 1000
qobj_list = []

if is_leakage:
    rb_fit_list = []
    for meas_x in range(2**nQ):
        rb_fit = rb.RBFitter(None, xdata, rb_opts['rb_pattern'])
        rb_fit_list.append(rb_fit)
else:
    rb_fit = rb.RBFitter(None, xdata, rb_opts['rb_pattern'])


for rb_seed,rb_circ_seed in enumerate(rb_circs):

    print('################### Compiling seed %d ###################'%rb_seed)
    
    if is_leakage:
        
        for meas_x in range(2**nQ):
            print("meas_x: %d"%meas_x)
            
            
            _rb_circ_seed_xmeas = []
            _len_v = len(length_vector)
            for i in range(_len_v):
                _rb_circ_seed_xmeas.append(rb_circ_seed[i][meas_x])
            
            new_rb_circ_seed = qiskit.compiler.transpile(_rb_circ_seed_xmeas, basis_gates=basis_gates)
            qobj = qiskit.compiler.assemble(new_rb_circ_seed, shots=shots)
            print('Simulating seed %d'%rb_seed)

            ## run on simulator 
            job = backend.run(qobj, noise_model=noise_leakage, backend_options={'max_parallel_experiments': 0})
            qobj_list.append(qobj)
            
            result = job.result()
            
            
#             # Add data to the fitter
#             rb_fit_list[meas_x].add_data(job.result())
            

            print('After seed %d, alpha: %f, EPC: %f'%(rb_seed, rb_fit_list[meas_x].fit[0]['params'][1], rb_fit_list[meas_x].fit[0]['epc']))
            

    else: 
        print(rb_circ_seed)
        new_rb_circ_seed = qiskit.compiler.transpile(rb_circ_seed, basis_gates=basis_gates)
        qobj = qiskit.compiler.assemble(new_rb_circ_seed, shots=shots)
        print('Simulating seed %d'%rb_seed)

        ## run on simulator 
        job = backend.run(qobj, noise_model=noise_model, backend_options={'max_parallel_experiments': 0})
        qobj_list.append(qobj)

        # Add data to the fitter
        _results = job.result()
        rb_fit.add_data(_results)
            
        print('After seed %d, alpha: %f, EPC: %f'%(rb_seed, rb_fit.fit[0]['params'][1], rb_fit.fit[0]['epc']))

## output 00

In [ ]:
# plt.figure(figsize=(8, 6))
# ax = plt.subplot(1, 1, 1)

# # Plot the essence by calling plot_rb_data
# rb_fit_list[0].plot_rb_data(0, ax=ax, add_label=True, show_plt=False)
    
# # Add title and label
# ax.set_title('%d Qubit RB'%(nQ), fontsize=18)

# plt.show()

## output 01

In [ ]:
plt.figure(figsize=(8, 6))
ax = plt.subplot(1, 1, 1)

# Plot the essence by calling plot_rb_data
rb_fit_list[1].plot_rb_data(0, ax=ax, add_label=True, show_plt=False)
    
# Add title and label
ax.set_title('%d Qubit RB'%(nQ), fontsize=18)

plt.show()

## output 10

In [ ]:
# plt.figure(figsize=(8, 6))
# ax = plt.subplot(1, 1, 1)

# # Plot the essence by calling plot_rb_data
# rb_fit_list[2].plot_rb_data(0, ax=ax, add_label=True, show_plt=False)
    
# # Add title and label
# ax.set_title('%d Qubit RB'%(nQ), fontsize=18)

# plt.show()

## output 11

In [ ]:
plt.figure(figsize=(8, 6))
ax = plt.subplot(1, 1, 1)

# Plot the essence by calling plot_rb_data
rb_fit_list[3].plot_rb_data(0, ax=ax, add_label=True, show_plt=False)
    
# Add title and label
ax.set_title('%d Qubit RB'%(nQ), fontsize=18)

plt.show()